<a href="https://colab.research.google.com/github/HJoon-K/Find_Dust/blob/yang/yjh.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import matplotlib
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder

from sklearn.model_selection import train_test_split, cross_val_score

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.naive_bayes import BernoulliNB, MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.cluster import KMeans
from sklearn.cluster import AgglomerativeClustering

from sklearn.metrics import accuracy_score, recall_score, precision_score
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.metrics import silhouette_score

from scipy.cluster.hierarchy import dendrogram
from scipy.cluster.hierarchy import linkage, fcluster

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
# 경고 메세지 감춤
import warnings
warnings.filterwarnings('ignore')

In [ ]:
dust16 = pd.read_excel('data/기간별_일평균_대기환경_정보_2016년.xlsx')
dust16

In [ ]:
dust16.info()

In [ ]:
dust16.drop('권역명', axis=1, inplace=True)
dust16.drop('오존(ppm)', axis=1, inplace=True)
dust16.drop('이산화질소농도(ppm)', axis=1, inplace=True)
dust16.drop('일산화탄소농도(ppm)', axis=1, inplace=True)
dust16.drop('아황산가스농도(ppm)', axis=1, inplace=True)
dust16

In [ ]:
dust16.columns = ['date', 'gu', 'PM10', 'PM25']
dust16

In [ ]:
dust16.to_csv('data/dust2016.csv', index=False)

In [ ]:
dust2016 = pd.read_csv('data/dust2016.csv')
dust2016

In [ ]:
dust17 = pd.read_excel('data/기간별_일평균_대기환경_정보_2017년.xlsx')
dust17

In [ ]:
dust17.info()

In [ ]:
dust17.drop('권역명', axis=1, inplace=True)
dust17.drop('오존(ppm)', axis=1, inplace=True)
dust17.drop('이산화질소농도(ppm)', axis=1, inplace=True)
dust17.drop('일산화탄소농도(ppm)', axis=1, inplace=True)
dust17.drop('아황산가스농도(ppm)', axis=1, inplace=True)
dust17

In [ ]:
dust17.columns = ['date', 'gu', 'PM10', 'PM25']
dust17

In [ ]:
dust17.to_csv('data/dust2017.csv', index=False)

In [ ]:
dust2017 = pd.read_csv('data/dust2017.csv')
dust2017

In [ ]:
dust18 = pd.read_csv('data/기간별_일평균_대기환경_정보_2018년.csv', encoding='euc-kr')
dust18

In [ ]:
dust18.info()

In [ ]:
dust18.drop('권역코드', axis=1, inplace=True)
dust18.drop('권역명', axis=1, inplace=True)
dust18.drop('측정소코드', axis=1, inplace=True)
dust18.drop('오존(ppm)', axis=1, inplace=True)
dust18.drop('이산화질소농도(ppm)', axis=1, inplace=True)
dust18.drop('일산화탄소농도(ppm)', axis=1, inplace=True)
dust18.drop('아황산가스농도(ppm)', axis=1, inplace=True)
dust18

In [ ]:
dust18.columns = ['date', 'gu', 'PM10', 'PM25']
dust18

In [ ]:
dust18.to_csv('data/dust2018.csv', index=False)

In [ ]:
dust2018 = pd.read_csv('data/dust2018.csv')
dust2018

In [ ]:
dust19 = pd.read_csv('data/기간별_일평균_대기환경_정보_2019년.csv', encoding='euc-kr')
dust19

In [ ]:
dust19.info()

In [ ]:
dust19.drop('권역코드', axis=1, inplace=True)
dust19.drop('권역명', axis=1, inplace=True)
dust19.drop('측정소코드', axis=1, inplace=True)
dust19.drop('오존(ppm)', axis=1, inplace=True)
dust19.drop('이산화질소농도(ppm)', axis=1, inplace=True)
dust19.drop('일산화탄소농도(ppm)', axis=1, inplace=True)
dust19.drop('아황산가스농도(ppm)', axis=1, inplace=True)
dust19

In [ ]:
dust19.columns = ['date', 'gu', 'PM10', 'PM25']
dust19

In [ ]:
dust19.to_csv('data/dust2019.csv', index=False)

In [ ]:
dust2019 = pd.read_csv('data/dust2019.csv')
dust2019

In [ ]:
dust20 = pd.read_csv('data/기간별_일평균_대기환경_정보_2020년.csv', encoding='euc-kr')
dust20

In [ ]:
dust20.info()

In [ ]:
dust20.drop('권역코드', axis=1, inplace=True)
dust20.drop('권역명', axis=1, inplace=True)
dust20.drop('측정소코드', axis=1, inplace=True)
dust20.drop('오존(ppm)', axis=1, inplace=True)
dust20.drop('이산화질소농도(ppm)', axis=1, inplace=True)
dust20.drop('일산화탄소농도(ppm)', axis=1, inplace=True)
dust20.drop('아황산가스농도(ppm)', axis=1, inplace=True)
dust20

In [ ]:
dust20.columns = ['date', 'gu', 'PM10', 'PM25']
dust20

In [ ]:
dust20.to_csv('data/dust2020.csv', index=False)

In [ ]:
dust2020 = pd.read_csv('data/dust2020.csv')
dust2020

In [ ]:
ra = pd.read_csv('data/도로현황(도로율).csv', header=1)
ra

In [ ]:
ra.info()

In [ ]:
ra.drop('자치구별(1)', axis=1, inplace=True)
ra.drop('도로연장 (km)', axis=1, inplace=True)
ra.columns = ['자치구별', '도로면적 (㎢)', '도로율 (%)']
ra.drop([0], axis=0, inplace=True)
ra

In [ ]:
ra.to_csv('data/road_area.csv', index=False)

In [ ]:
road_area = pd.read_csv('data/road_area.csv')
road_area

In [ ]:
p = pd.read_csv('data/서울시 주요 공원현황.csv', encoding='euc-kr')
p

In [ ]:
p.info()

In [ ]:
p.drop(['연번', '공원개요', '면적', '개원일', '주요시설',
        '주요식물', '안내도', '오시는길', '이용시참고사항',
        '이미지', '전화번호', '관리부서', 'X좌표(GRS80TM)', 'Y좌표(GRS80TM)', 'X좌표(WGS84)', 'Y좌표(WGS84)', '바로가기'], axis=1, inplace=True)
p

In [ ]:
p.to_csv('data/park.csv', index=False)

In [ ]:
park = pd.read_csv('data/park.csv')
park